In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'e:\\TextSummarization'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class modelevaluationconfig:
    root_dir: Path
    model_path: Path
    tokenizer_path: Path
    data_path: Path
    metric_file_name: Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    
    def __init__(
        self,
        config_path = CONFIG_FILE_PATH,
        params_path = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> modelevaluationconfig:
        
        config = self.config.model_evaluation
        
        model_evaluation_config = modelevaluationconfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name,
            data_path=config.data_path
        )
        
        return model_evaluation_config

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
class modelevaluation:

    def __init__(self, config: modelevaluationconfig):
        
        self.config = config
        
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):

        """split the dataset into smaller chunks so that we can process simultaneously
            yield successive batch-sized chunks from list_of_elements"""
        for ind in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[ind : ind + batch_size]
            
    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                                                                    batch_size=16, device=device,
                                                                    column_text='article',
                                                                    column_summary='highlights'):

        article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024, truncation=True, padding='max_length', return_tensors='pt')
            summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                                                                    attention_mask=inputs['attention_mask'].to(device),
                                                                    length_penalty=0.8, num_beams=8, max_length=128)

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                                                clean_up_tokenization_spaces=True) for s in summaries]
            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score
        
    def evluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_progress - AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        
        rouge_metric = load_metric('rouge')
        
        score = calculate_metric_on_test_ds(dataset_samsum['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size=2, device=device,
                                    column_text='dialogue', column_summary='summary')

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=[f'pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

In [ ]:
config = ConfigurationManager()
modelevaluation_config = config.get_model_evaluation_config()
modelevaluation = modelevaluation(config=modelevaluation_config)
modelevaluation.evaluate()